In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from diffusers import DDPMPipeline, DDPMScheduler, UNet2DModel
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
from tqdm import tqdm
from copy import deepcopy
from torch.optim import AdamW
from torch.distributions import Laplace
import numpy as np
import random

# --- Classe Principale Laplacian DDPM ---

class LaplacianDDPM:
    def __init__(self, model_id, device="cuda"):
        self.device = device
        self.model_id = model_id

        # Chargement du modèle
        self.pipeline = DDPMPipeline.from_pretrained(model_id)
        self.unet = self.pipeline.unet.to(device)
        self.scheduler = self.pipeline.scheduler

        # Configuration par défaut
        self.config = {
            "layer_name": "mid_block.resnets.0.conv1.weight",  # Couche cible
            "watermark_size": 32,       # Nombre de bits du watermark
            "spreading_factor": 10,     # Facteur d'étalement (S)
            "variance": 1,            # Variance pour la distribution Laplacienne
            "lr": 1e-4,
            "epochs": 30
        }

        self.saved_keys = {}

    def _get_target_layer(self, model):
        """Récupère le tenseur des poids de la couche cible."""
        for name, param in model.named_parameters():
            if name == self.config["layer_name"]:
                return param
        raise ValueError(f"Paramètre {self.config['layer_name']} introuvable.")

    @staticmethod
    def _get_rand_bits(size):
        """Génère une séquence binaire aléatoire."""
        return [random.randint(0, 1) for _ in range(size)]

    def keygen(self, watermark_size):
        """
        Génère le watermark et les signes correspondants.
        Returns:
            wat_signs: Watermark en {-1, +1}
            watermark: Watermark binaire en {0, 1}
        """
        watermark = torch.tensor(self._get_rand_bits(watermark_size)).float().to(self.device)
        wat_signs = torch.tensor([-1.0 if i == 0 else 1.0 for i in watermark], dtype=torch.float).to(self.device)
        return wat_signs, watermark

    def generate_laplacian_pseudo_sequence(self, s, l, loc=0.0, scale=1.0, seed=None):
        """
        Génère une séquence pseudo-aléatoire avec distribution Laplacienne.
        
        Args:
            s: Spreading factor
            l: Taille du watermark
            loc: Paramètre de localisation (moyenne)
            scale: Paramètre d'échelle (diversité)
            seed: Graine aléatoire pour la reproductibilité
        
        Returns:
            torch.Tensor: Séquence pseudo de taille (s * l,)
        """
        if seed is not None:
            torch.manual_seed(seed)
            np.random.seed(seed)

        total_size = s * l
        laplace_dist = Laplace(loc=torch.tensor(loc), scale=torch.tensor(scale))
        pseudo_seq = laplace_dist.sample((total_size,)).to(self.device)
        
        return pseudo_seq

    def generate_weight_sequence(self, watermark_signs, pseudo_seq, S):
        """
        Génère la séquence de poids avec le facteur d'étalement S.
        wm_j = u_i · s_j
        """
        l = len(watermark_signs)
        n = len(pseudo_seq)

        if n < l * S:
            raise ValueError(f"Spreading sequence length ({n}) must be >= l*S ({l * S})")

        weight_seq = torch.zeros(l * S, dtype=torch.float32).to(self.device)

        for i in range(1, l + 1):
            start_idx = (i - 1) * S
            end_idx = i * S

            for j in range(start_idx, end_idx):
                weight_seq[j] = watermark_signs[i - 1] * pseudo_seq[j]

        return weight_seq

    def replace_weights(self, model, weight_sequence, selected_indices=None):
        """
        Remplace les poids sélectionnés avec les valeurs du watermark.
        """
        n = len(weight_sequence)
        target_layer = self._get_target_layer(model)
        
        weight_tensor = target_layer.data
        original_shape = weight_tensor.shape
        flat_weights = weight_tensor.flatten()
        total_weights = flat_weights.numel()

        if n > total_weights:
            raise ValueError(f"Sequence length {n} exceeds total weights {total_weights}")

        if selected_indices is None:
            selected_indices = random.sample(range(total_weights), n)

        for i, idx in enumerate(selected_indices):
            flat_weights[idx] = weight_sequence[i]

        target_layer.data = flat_weights.reshape(original_shape)
        return selected_indices

    def pick_weights_by_indices(self, model, selected_indices):
        """
        Extrait les poids sélectionnés d'une couche.
        """
        target_layer = self._get_target_layer(model)
        flat_weights = target_layer.data.flatten()
        selected_weights = flat_weights[selected_indices]
        return selected_weights

    def calculate_watermark_bits(self, selected_weights, spreading_sequence, S):
        """
        Calcule les bits du watermark à partir des poids extraits.
        """
        watermark_length = len(spreading_sequence) // S

        if len(spreading_sequence) != watermark_length * S:
            raise ValueError(f"Spreading sequence length must be divisible by S")

        watermark_bits = torch.zeros(watermark_length, dtype=torch.float32).to(self.device)
        correlation_sums = torch.zeros(watermark_length, dtype=torch.float32).to(self.device)

        for i in range(1, watermark_length + 1):
            start_idx = (i - 1) * S
            end_idx = i * S

            s_slice = spreading_sequence[start_idx:end_idx]
            w_slice = selected_weights[start_idx:end_idx]

            correlation_sum = torch.sum(s_slice * w_slice)
            correlation_sums[i - 1] = correlation_sum

            bit_value = 1.0 if correlation_sum >= 0 else 0.0
            watermark_bits[i - 1] = bit_value

        return watermark_bits, correlation_sums

    @staticmethod
    def _compute_ber(extracted, target):
        """Calcule le Bit Error Rate."""
        return (extracted != target).float().mean().item()

    def embed(self, dataloader):
        """
        Incorpore le watermark Laplacien pendant le finetuning.
        """
        print(f"--- Démarrage Embedding LAPLACIAN ({self.config['layer_name']}) ---")
        print(f"Watermark size: {self.config['watermark_size']} bits")
        print(f"Spreading factor: {self.config['spreading_factor']}")

        watermarked_unet = self.unet
        watermarked_unet.train()

        # 1. Génération des clés
        watermark_signs, watermark = self.keygen(self.config["watermark_size"])
        
        # Calcul de gamma (scale parameter)
        gamma = self.config["variance"] / np.sqrt(2)
        
        # Génération de la séquence pseudo-aléatoire Laplacienne
        pseudo_seq = self.generate_laplacian_pseudo_sequence(
            s=self.config['spreading_factor'],
            l=self.config['watermark_size'],
            loc=0.0,
            scale=gamma
        )
        
        # Génération de la séquence de poids
        weight_seq = self.generate_weight_sequence(
            watermark_signs=watermark_signs,
            pseudo_seq=pseudo_seq,
            S=self.config['spreading_factor']
        )
        
        print(f"Weight sequence length: {len(weight_seq)}")

        # 2. Remplacement initial des poids
        selected_indices = self.replace_weights(watermarked_unet, weight_seq)
        print(f"Selected {len(selected_indices)} weight positions")

        # 3. Optimiseur
        optimizer = torch.optim.AdamW(watermarked_unet.parameters(), lr=self.config["lr"])
        mse_loss = nn.MSELoss()

        # 4. Boucle d'entraînement
        for epoch in range(self.config["epochs"]):
            pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}/{self.config['epochs']}")
            
            for clean_images, _ in pbar:
                clean_images = clean_images.to(self.device)
                bs = clean_images.shape[0]

                # A. Processus de Diffusion (Forward)
                noise = torch.randn_like(clean_images).to(self.device)
                timesteps = torch.randint(
                    0, self.scheduler.config.num_train_timesteps, 
                    (bs,), device=self.device
                ).long()
                noisy_images = self.scheduler.add_noise(clean_images, noise, timesteps)

                optimizer.zero_grad()

                # B. Prédiction (Task Loss)
                noise_pred = watermarked_unet(noisy_images, timesteps).sample
                l_main = mse_loss(noise_pred, noise)

                l_main.backward()
                optimizer.step()

                # C. Re-application du watermark (freeze weights)
                self.replace_weights(
                    model=watermarked_unet,
                    weight_sequence=weight_seq,
                    selected_indices=selected_indices
                )

                # D. Calcul du BER
                extracted_weights = self.pick_weights_by_indices(watermarked_unet, selected_indices)
                extracted_wm, _ = self.calculate_watermark_bits(
                    extracted_weights, pseudo_seq, self.config['spreading_factor']
                )
                ber = self._compute_ber(extracted_wm, watermark)

                pbar.set_postfix(L_Main=f"{l_main.item():.4f}", BER=f"{ber:.4f}")

        # Sauvegarde des clés
        self.saved_keys = {
            "watermark": watermark,
            "watermark_signs": watermark_signs,
            "pseudo_seq": pseudo_seq,
            "weight_seq": weight_seq,
            "selected_indices": selected_indices,
            "watermarked_unet": watermarked_unet
        }
        
        print(f"\n✅ Embedding terminé! BER final: {ber:.4f}")
        return watermarked_unet

    def extract(self, suspect_unet=None):
        """
        Extrait le watermark d'un modèle suspect.
        """
        if suspect_unet is None:
            suspect_unet = self.saved_keys["watermarked_unet"]

        watermark = self.saved_keys["watermark"]
        pseudo_seq = self.saved_keys["pseudo_seq"]
        selected_indices = self.saved_keys["selected_indices"]

        # 1. Extraction des poids
        try:
            extracted_weights = self.pick_weights_by_indices(suspect_unet, selected_indices)
        except ValueError:
            print("⚠️ Couche cible introuvable dans le modèle suspect.")
            return 1.0, None

        # 2. Calcul du watermark extrait
        extracted_wm, correlation_sums = self.calculate_watermark_bits(
            extracted_weights, pseudo_seq, self.config['spreading_factor']
        )

        # 3. Calcul du BER
        ber = self._compute_ber(extracted_wm, watermark)

        print(f"BER Extrait: {ber:.4f}")
        return ber, extracted_wm


/home/latim/PycharmProjects/WatDNN/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# --- EXEMPLE D'EXÉCUTION ---

# 1. Setup Data
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize((0.5,), (0.5,))
])
dataset = datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# 2. Embedding Laplacian
laplacian_defense = LaplacianDDPM("google/ddpm-cifar10-32")

# Configuration optionnelle
laplacian_defense.config.update({
    "watermark_size": 32,
    "spreading_factor": 10,
    "variance": 1, #0.025277,
    "epochs": 30,
    "lr": 1e-4
})

watermarked_model = laplacian_defense.embed(dataloader)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.
Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80: Error no file named diffusion_pytorch_model.safetensors found in directory /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00,  2.45it/s]


--- Démarrage Embedding LAPLACIAN (mid_block.resnets.0.conv1.weight) ---
Watermark size: 32 bits
Spreading factor: 10
Weight sequence length: 320
Selected 320 weight positions


Epoch 30/30: 100%|██████████| 782/782 [01:03<00:00, 12.40it/s, BER=0.0000, L_Main=0.0309]


✅ Embedding terminé! BER final: 0.0000


In [3]:
# 3. Test d'extraction sur le modèle watermarké
print("\n--- Test Extraction sur modèle watermarké ---")
ber, extracted_wm = laplacian_defense.extract()
print(f"Watermark original: {laplacian_defense.saved_keys['watermark'][:10]}...")
print(f"Watermark extrait:  {extracted_wm[:10]}...")


--- Test Extraction sur modèle watermarké ---
BER Extrait: 0.0000
Watermark original: tensor([0., 1., 1., 0., 0., 1., 1., 0., 0., 1.], device='cuda:0')...
Watermark extrait:  tensor([0., 1., 1., 0., 0., 1., 1., 0., 0., 1.], device='cuda:0')...


In [4]:
# --- Fonction de Distillation (Attaque) ---

def run_distillation_attack_laplacian(laplacian_obj, dataloader, epochs=5, lr=1e-4):
    """
    Tente de transférer la fonctionnalité du modèle Laplacian vers un modèle vierge.
    Vérifie si la marque (basée sur les poids) survit.
    """
    device = laplacian_obj.device

    # 1. Teacher (Gelé)
    teacher_unet = laplacian_obj.saved_keys["watermarked_unet"]
    teacher_unet.eval()
    for p in teacher_unet.parameters():
        p.requires_grad = False

    # 2. Student (Vierge - Même architecture)
    print("\n--- Initialisation du Student ---")
    student_pipeline = DDPMPipeline.from_pretrained("google/ddpm-cifar10-32")
    student_unet = student_pipeline.unet.to(device)
    student_unet.train()

    # Sanity Checks
    teacher_ber, _ = laplacian_obj.extract(teacher_unet)
    student_ber, _ = laplacian_obj.extract(student_unet)
    print(f"[Check] BER Teacher: {teacher_ber:.4f}")
    print(f"[Check] BER Student (Avant): {student_ber:.4f}")

    optimizer = AdamW(student_unet.parameters(), lr=lr)
    noise_scheduler = laplacian_obj.scheduler
    history = {"loss": [], "ber": []}

    print(f"\n--- Distillation Laplacian ({epochs} epochs) ---")

    for epoch in range(epochs):
        pbar = tqdm(dataloader, desc=f"Epoch {epoch+1}")
        running_loss = 0.0

        for clean_images, _ in pbar:
            clean_images = clean_images.to(device)
            bs = clean_images.shape[0]

            # A. Input Noise
            noise = torch.randn_like(clean_images).to(device)
            timesteps = torch.randint(
                0, noise_scheduler.config.num_train_timesteps, 
                (bs,), device=device
            ).long()
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)

            # B. Distillation (Output Matching)
            with torch.no_grad():
                target_pred = teacher_unet(noisy_images, timesteps).sample

            student_pred = student_unet(noisy_images, timesteps).sample

            loss = F.mse_loss(student_pred, target_pred)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            pbar.set_postfix(Loss=loss.item())

        # C. Vérification du BER après chaque epoch
        current_ber, _ = laplacian_obj.extract(student_unet)
        history["ber"].append(current_ber)
        history["loss"].append(running_loss / len(dataloader))

        print(f"👉 Fin Epoch {epoch+1} | Loss: {loss.item():.4f} | BER Student: {current_ber:.4f}")

    return student_unet, history

In [ ]:
# 4. Attaque par Distillation
student_model, attack_stats = run_distillation_attack_laplacian(
    laplacian_defense, 
    dataloader, 
    epochs=1000
)

Cannot initialize model with low cpu memory usage because `accelerate` was not found in the environment. Defaulting to `low_cpu_mem_usage=False`. It is strongly recommended to install `accelerate` for faster and less memory-intense model loading. You can do so with: 
```
pip install accelerate
```
.



--- Initialisation du Student ---


Loading pipeline components...:   0%|          | 0/2 [00:00<?, ?it/s]An error occurred while trying to fetch /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80: Error no file named diffusion_pytorch_model.safetensors found in directory /home/latim/.cache/huggingface/hub/models--google--ddpm-cifar10-32/snapshots/267b167dc01f0e4e61923ea244e8b988f84deb80.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
Loading pipeline components...: 100%|██████████| 2/2 [00:00<00:00, 21.51it/s]


BER Extrait: 0.0000
BER Extrait: 0.3750
[Check] BER Teacher: 0.0000
[Check] BER Student (Avant): 0.3750

--- Distillation Laplacian (1000 epochs) ---


Epoch 1: 100%|██████████| 782/782 [01:18<00:00, 10.01it/s, Loss=0.000204]


...

Epoch 47: 100%|██████████| 782/782 [01:31<00:00,  8.54it/s, Loss=4.47e-5] 


BER Extrait: 0.0312
👉 Fin Epoch 47 | Loss: 0.0000 | BER Student: 0.0312


Epoch 48: 100%|██████████| 782/782 [01:33<00:00,  8.38it/s, Loss=6.48e-5] 


BER Extrait: 0.0312
👉 Fin Epoch 48 | Loss: 0.0001 | BER Student: 0.0312


Epoch 49:   9%|▊         | 68/782 [00:08<01:21,  8.72it/s, Loss=5.73e-5] 

In [ ]:
# 5. Visualisation des résultats
import matplotlib.pyplot as plt

fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# BER over epochs
axes[0].plot(attack_stats["ber"], marker='o')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('BER')
axes[0].set_title('BER Evolution During Distillation Attack')
axes[0].axhline(y=0.5, color='r', linestyle='--', label='Random guess (0.5)')
axes[0].legend()
axes[0].grid(True)

# Loss over epochs
axes[1].plot(attack_stats["loss"], marker='s', color='orange')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Loss')
axes[1].set_title('Distillation Loss Over Epochs')
axes[1].grid(True)

plt.tight_layout()
plt.show()

In [ ]:
# 6. Comparaison finale
print("\n" + "="*50)
print("RÉSUMÉ FINAL")
print("="*50)

teacher_ber, _ = laplacian_defense.extract(laplacian_defense.saved_keys["watermarked_unet"])
student_ber, _ = laplacian_defense.extract(student_model)

print(f"\nBER Teacher (watermarké): {teacher_ber:.4f}")
print(f"BER Student (après distillation): {student_ber:.4f}")



In [ ]:
%%sql


In [ ]:
%%sql
